In [ ]:
%%capture # hide output
%pip install pandas
%pip install torch
%pip install scikit-learn

In [ ]:
import sklearn
import torch
import torch.nn as nn
import torch.nn.functional as f
import pandas as pd
from sklearn.model_selection import train_test_split

KeyboardInterrupt: 

In [ ]:
class Model(nn.Module):
    def __init__(self, in_features=9, h1=8, h2=8, out_features=2):
        super().__init__()
        self.fc1 = nn.Linear(in_features, h1)
        self.fc2 = nn.Linear(h1, h2)
        self.out = nn.Linear(h2, out_features)
    def forward(self, x):
        x = f.relu(self.fc1(x))
        x = f.relu(self.fc2(x))
        x = f.relu(self.out(x))
        return x

In [ ]:
torch.manual_seed(67)
model = Model()

dataset_path = "instafake-dataset-master/data/automated-v1.0/"

automated_dataset = pd.read_json(dataset_path + "automatedAccountData.json")
non_automated_dataset = pd.read_json(dataset_path + "nonautomatedAccountData.json")

In [ ]:
combined = pd.concat([automated_dataset, non_automated_dataset], axis=0)

In [ ]:
combined

,userMediaCount,mediaLikeNumbers,mediaCommentNumbers,mediaCommentsAreDisabled,mediaHashtagNumbers,mediaUploadTimes,mediaHasLocationInfo,userFollowerCount,userFollowingCount,userHasHighlighReels,userHasExternalUrl,userTagsCount,userBiographyLength,usernameLength,usernameDigitCount,automatedBehaviour
0,97,"[100, 78, 112, 78, 77, 111, 97, 95, 63, 83, 42...","[5, 2, 10, 0, 3, 2, 2, 2, 4, 0, 0, 6, 6, 0, 2,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[10, 10, 12, 10, 13, 13, 10, 12, 10, 11, 11, 1...","[1540847457, 1540744841, 1538410400, 153729214...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",1612,7108,0,1,36,133,12,0,1
1,863,"[229, 331, 180, 326, 313, 157, 245, 161, 182, ...","[3, 2, 3, 4, 7, 2, 0, 6, 2, 2, 1, 9, 2, 0, 2, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[8, 11, 14, 17, 12, 10, 13, 14, 14, 11, 10, 11...","[1542664246, 1542569444, 1542564418, 154211928...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",3028,912,1,0,77,99,11,0,1
2,149,"[237, 254, 350, 257, 180, 172, 141, 84, 218, 8...","[12, 16, 37, 11, 3, 4, 2, 4, 5, 1, 1, 2, 3, 3,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[25, 25, 25, 25, 15, 15, 15, 5, 15, 10, 10, 10...","[1543220394, 1542972220, 1542868998, 154278889...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, ...",5213,1157,1,1,56,120,18,0,1
3,33,"[163, 30, 92, 111, 145, 110, 81, 81, 61, 60, 6...","[2, 2, 7, 14, 17, 3, 2, 5, 2, 1, 5, 5, 8, 1, 4...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 1, 0, 0, 0, 0, 2, 0, 0, 0, 0, 14, 14, 0, 0...","[1543886933, 1543793413, 1543704343, 154344770...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",12423,1763,0,0,2,55,13,0,1
4,1000,"[61, 222, 437, 829, 681, 440, 920, 256, 1017, ...","[4, 0, 13, 9, 9, 6, 16, 4, 25, 6, 3, 4, 16, 10...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[4, 1, 2, 3, 3, 0, 0, 1, 0, 2, 1, 2, 3, 1, 1, ...","[1543939920, 1543699089, 1543187756, 154264414...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, ...",9496,2412,1,0,91,33,8,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
695,26,"[117, 82, 70, 50, 94, 45, 40, 47, 44, 43, 45, ...","[0, 1, 0, 0, 4, 0, 0, 2, 1, 0, 5, 6, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 0, 0, 1, 0, 2, 1, 0, 1, 1, 0, 0, 1, 0, 2, ...","[1533665377, 1529705967, 1525534881, 145340569...","[1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, ...",303,335,0,0,23,22,12,0,0
696,12,"[86, 87, 75, 24, 30, 29, 22, 33, 34, 29, 17, 13]","[4, 5, 10, 0, 0, 1, 1, 2, 0, 0, 3, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]","[1541858886, 1539619797, 1537199999, 146400347...","[1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0]",270,318,0,0,10,23,11,0,0
697,11,"[28, 18, 20, 15, 8, 13, 17, 15, 10, 4, 11]","[0, 0, 0, 0, 0, 0, 0, 2, 2, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[2, 10, 6, 0, 3, 2, 6, 4, 0, 2, 0]","[1417601401, 1416411495, 1412454197, 140580953...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",175,179,0,0,4,6,5,0,0
698,116,"[146, 132, 123, 148, 122, 118, 130, 144, 119, ...","[10, 0, 0, 0, 2, 2, 4, 9, 4, 0, 4, 3, 3, 3, 3,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1540144489, 1534530033, 1532627229, 153228440...","[1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, ...",311,311,0,0,13,0,10,0,0


In [ ]:
X = combined.drop(["automatedBehaviour", "mediaLikeNumbers", "mediaCommentNumbers", "mediaCommentsAreDisabled", "mediaHashtagNumbers", "mediaUploadTimes", "mediaHasLocationInfo"], axis=1)
y = combined["automatedBehaviour"]
X = X.values
y = y.values

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=67)

In [ ]:
X_train = torch.FloatTensor(X_train)
X_test = torch.FloatTensor(X_test)
y_train = torch.LongTensor(y_train)
y_test = torch.LongTensor(y_test)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0005)

In [ ]:
epochs = 5000
losses = []
for i in range(epochs):
    y_pred = model.forward(X_train)
    loss = criterion(y_pred, y_train)
    losses.append(loss.detach().numpy())

    if i % 10 == 0:
        print(f"{i}) Loss = {loss}")

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

0) Loss = 0.5023263096809387
10) Loss = 0.5020067095756531
20) Loss = 0.501615583896637
30) Loss = 0.5013132095336914
40) Loss = 0.5007638335227966
50) Loss = 0.50003582239151
60) Loss = 0.49954330921173096
70) Loss = 0.49913957715034485
80) Loss = 0.4987347722053528
90) Loss = 0.49831652641296387
100) Loss = 0.49795764684677124
110) Loss = 0.49751532077789307
120) Loss = 0.49718356132507324
130) Loss = 0.49681171774864197
140) Loss = 0.49658557772636414
150) Loss = 0.49626991152763367
160) Loss = 0.4959861934185028
170) Loss = 0.4957828223705292
180) Loss = 0.49544674158096313
190) Loss = 0.4952164590358734
200) Loss = 0.49485915899276733
210) Loss = 0.4945104420185089
220) Loss = 0.4940263032913208
230) Loss = 0.4936576187610626
240) Loss = 0.49336957931518555
250) Loss = 0.49307799339294434
260) Loss = 0.4926357865333557
270) Loss = 0.4917815923690796
280) Loss = 0.4911724030971527
290) Loss = 0.4906264543533325
300) Loss = 0.4901741147041321
310) Loss = 0.48977968096733093
320) Los